In [ ]:
import json
import sys
from os import listdir
from os.path import isfile, join
import re
import string
import pandas as pd
import numpy as np

import time
from tqdm import tqdm
from typing import List

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
# from keras.preprocessing.sequence import pad_sequences

import transformers
from transformers import AutoTokenizer, AutoModel, utils
from transformers import AutoTokenizer, AutoModelForSequenceClassification,Trainer, TrainingArguments
from datasets import Dataset
transformers.logging.set_verbosity_error()
utils.logging.set_verbosity_error()  # Suppress standard warnings

from bertviz import model_view, head_view


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


import eli5
from eli5.lime import TextExplainer
# from captum.attr import IntegratedGradients
# import matplotlib.pyplot as plt


from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt

## Data

In [ ]:
sen_train_data = pd.read_csv('../sentence_split/train_sentence_data.csv')
sen_test_data = pd.read_csv('../sentence_split/test_sentence_data.csv')
sen_val_data = pd.read_csv('../sentence_split/val_sentence_data.csv')

In [ ]:
doc_train_data = pd.read_csv("../complete_sentence/train_processed_data.csv")
doc_test_data = pd.read_csv("../complete_sentence/train_processed_data.csv") 

In [ ]:
sen_train_data = sen_train_data.drop(['File_id'],axis=1)
sen_test_data = sen_test_data.drop(['File_id'],axis=1)
sen_val_data = sen_val_data.drop(['File_id'],axis=1)

In [ ]:
# def get_20(data):
#     data_1 = data.loc[data['Status'] ==1].iloc[:20]
#     data_2 = data.loc[data['Status'] ==0].iloc[:20]
#     frames = [data_1, data_2]
#     return pd.concat(frames)

# sen_train_data = get_20(sen_train_data) 
# sen_test_data = get_20(sen_test_data)
# sen_val_data = get_20(sen_val_data)

In [ ]:
doc_train_text = doc_train_data.Paper_text.values.tolist()
doc_train_status = doc_train_data.Status.values.tolist()

doc_test_text = doc_test_data.Paper_text.values.tolist()
doc_test_status = doc_test_data.Status.values.tolist()

In [ ]:
sen_train_data=sen_train_data.sample(frac = 1)
sen_test_data=sen_test_data.sample(frac = 1)
sen_val_data=sen_val_data.sample(frac = 1)

In [ ]:
sen_train_data.to_csv('Train_auto.csv',index=False)
sen_test_data.to_csv('Test_auto.csv',index=False)
sen_val_data.to_csv('val_auto.csv',index=False)

In [ ]:
sen_train_data.Status.value_counts()

In [ ]:
# Load train and validation datasets from CSV files
train_dataset = Dataset.from_csv('Train_auto.csv')
val_dataset = Dataset.from_csv('val_auto.csv')
test_dataset = Dataset.from_csv('Test_auto.csv')

# Rename the columns to 'text' and 'label' to match the expected format for sequence classification
train_dataset = train_dataset.rename_column('Sentence', 'text').rename_column('Status', 'label')
val_dataset = val_dataset.rename_column('Sentence', 'text').rename_column('Status', 'label')
test_dataset = test_dataset.rename_column('Sentence', 'text').rename_column('Status', 'label')


In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/allenai-specter')

# Define a function to tokenize the text and create input sequences
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True,max_length=512)

# Apply the tokenization function to the train and validation datasets
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/allenai-specter', num_labels=2)

In [ ]:
# Train the model
training_args = TrainingArguments(
    output_dir='./results/specter',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [ ]:
trainer = Trainer(
    model=model,                     # the instantiated Transformers model to be trained
    args=training_args,              # training arguments, defined above
    train_dataset=train_dataset,     # training dataset
    eval_dataset=val_dataset,        # evaluation dataset
)
trainer.train()

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate(test_dataset)
print(eval_results)

In [ ]:
def model_adapter(texts: List[str]):
    
    all_scores = []

    for i in tqdm(range(0, len(texts), 64)):

        batch = texts[i:i+64]
        #print(batch)
        
        # use bert encoder to tokenize text 
        encoded_input = tokenizer(batch, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=model.config.max_position_embeddings-2)

        # run the model
        output = model(**encoded_input)
        #print(output)
        # by default this model gives raw logits rather 
        # than a nice smooth softmax so we apply it ourselves here
        scores = output[0].softmax(1).detach().numpy()
        #print(scores)

        all_scores.extend(scores)

    return np.array(all_scores)

In [ ]:
sen = sen_test_data.Sentence
lab = sen_test_data.Status

In [ ]:
specter_exp = TextExplainer(n_samples=5000, random_state=42)
specter_exp.fit(sen[0], model_adapter)
specter_exp.explain_prediction(target_names=list(model.config.id2label.values()))

In [ ]:
weights = specter_exp.explain_weights(top=None)


In [ ]:
weights

In [ ]:
weights = specter_exp.explain_weights()

sen_positivie_words= {}

for i in weights.targets[0].feature_weights.pos:
    #print(i.feature)
    g = sen_positivie_words.get(i.feature,-1)
    if g==-1:
        sen_positivie_words[i.feature]=1
    else:
        sen_positivie_words[i.feature]+=1
        

sen_negative_words= {}

for i in weights.targets[0].feature_weights.neg:
    #print(i.feature)
    g = sen_negative_words.get(i.feature,-1)
    if g==-1:
        sen_negative_words[i.feature]=1
    else:
        sen_negative_words[i.feature]+=1

        
wc = WordCloud(background_color="white",width=1000,height=1000,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(sen_positivie_words)
plt.imshow(wc)


In [ ]:
wc = WordCloud(background_color="white",width=1000,height=1000,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(sen_negative_words)
plt.imshow(wc)

## very inportant

In [ ]:
# print(weights.targets[0].feature_weights.pos[0].weight,weights.targets[0].feature_weights.pos[0].feature)

In [ ]:
# d = specter_exp.explain_prediction()

### Logistic Regression

In [ ]:
doc_train_data = pd.read_csv("./complete_sentence/train_processed_data.csv")
doc_test_data = pd.read_csv("./complete_sentence/train_processed_data.csv") 

In [ ]:
doc_train_data = get_20(doc_train_data)
doc_test_data = get_20(doc_test_data)

In [ ]:
doc_train_data.head()

In [ ]:
doc_train_text = doc_train_data.Paper_text.values.tolist()
doc_train_status = doc_train_data.Status.values.tolist()

doc_test_text = doc_test_data.Paper_text.values.tolist()
doc_test_status = doc_test_data.Status.values.tolist()

In [ ]:
class Padding(BaseEstimator, TransformerMixin):
    
    def __init__(self,model,tokenizer):
        self.model = model # Configure model to return attention values
        self.tokenizer = tokenizer
        self.mxlenght = 400
        print('\n>>>>>>>init() called.\n')

    def fit(self, X, y = None):
        print('\n>>>>>>>fit() called.\n')
        return self
    

    def transform(self, X, y = None):
        embeddings = []
        for i in tqdm(X):
            temp = [] 
            sentence_list = i.split(".")
            for i in sentence_list:
                if len(i)==0:
                    continue
                encoded_input = tokenizer(i,return_tensors='pt', padding=True, truncation=True,max_length=model.config.max_position_embeddings-2)
                output = model(**encoded_input)
                pred = np.argmax(output[0].softmax(1).detach().numpy())
                temp.append(pred)
            size = self.mxlenght - len(temp)
            if size > 0:
                temp.extend([-1]*size)
            elif size < 0:
                temp = temp[0:self.mxlenght]
            else:
                pass
            embeddings.append(temp)
        return embeddings

In [ ]:
pipe1 = Pipeline(steps=[
                       ('Documnet Embeddings', Padding(model,tokenizer)), # this will trigger a call to __init__
                       ('Logistic Regression', LogisticRegression(solver='lbfgs')),

])

pipe1.fit(doc_train_text, doc_train_status)
pipe1.score(doc_test_text,doc_test_status)

In [ ]:
target =['Reject','Accept']
doc = doc_test_text[0]
pipe1_exp = TextExplainer(n_samples=10,random_state=42)
pipe1_exp.fit(doc, pipe1.predict_proba)
pipe1_exp.show_prediction(target_names= target)

In [ ]:
weights = pipe1_exp.explain_weights(top= None)
positivie_words= {}

for i in weights.targets[0].feature_weights.pos:
    #print(i.feature)
    g = positivie_words.get(i.feature,-1)
    if g==-1:
        positivie_words[i.feature]=1
    else:
        positivie_words[i.feature]+=1
        

negative_words= {}

for i in weights.targets[0].feature_weights.neg:
    #print(i.feature)
    g = negative_words.get(i.feature,-1)
    if g==-1:
        negative_words[i.feature]=1
    else:
        negative_words[i.feature]+=1
# print(weights.targets[0].feature_weights.pos[0].weight,weights.targets[0].feature_weights.pos[0].feature)

In [ ]:
wc = WordCloud(background_color="white",width=1000,height=1000,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(positivie_words)
plt.imshow(wc)

In [ ]:
wc = WordCloud(background_color="white",width=1000,height=1000,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(negative_words)
plt.imshow(wc)

In [ ]:
def print_prediction(doc):
    #print(doc)
    y_pred = pipe1.predict_proba([doc])[0]
    tar =['Reject','Accept']
    for target, prob in zip(tar, y_pred):
        print("{:.3f} {}".format(prob, target))

In [ ]:
print_prediction(doc_test_text[0])

## SVM CLASSIFIER

In [ ]:
class CustomEmbedding(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.model_name = "sentence-transformers/bert-base-nli-mean-tokens"
        self.model = AutoModel.from_pretrained(self.model_name, output_attentions=True)  # Configure model to return attention values
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        print('\n>>>>>>>init() called.\n')

    def fit(self, X, y = None):
        print('\n>>>>>>>fit() called.\n')
        return self
    
    def mean_pooling(self,model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    def bert(self,text):
        encoded_input = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        inputs = self.tokenizer.encode(text, return_tensors='pt')  # Tokenize input text

        outputs = self.model(inputs)  # Run model
        attention = outputs[-1]  # Retrieve attention from model outputs

        tokens = self.tokenizer.convert_ids_to_tokens(inputs[0]) 
        sentence_embeddings = self.mean_pooling(outputs, encoded_input['attention_mask'])
        return sentence_embeddings.detach().numpy()[0].tolist()

    def transform(self, X, y = None):
        embeddings = []
        for i in X:
            emb = self.bert(i)
            embeddings.append(emb)
        #print(embeddings)
        return embeddings
        

In [ ]:
pipe2 = Pipeline(steps=[
                       ('Bert Embeddings', CustomEmbedding()),    # this will trigger a call to __init__
                       ('Support Vector Classifier', SVC(kernel='rbf',probability=True))
])


In [ ]:
sen_text = sen_train_data['Sentence']
sen_label = sen_train_data['Status']
pipe2.fit(sen_text, sen_label)

In [ ]:
sen_test_text = sen_test_data['Sentence']
sen_test_lab = sen_test_data['Status']
pipe2.score(sen_test_text,sen_test_lab)

In [ ]:
target =['Reject','Accept']
doc = 'however, models that make use of this strategy eventually fail after a certain level of complexity (e'
pipe2_exp = TextExplainer(random_state=42)
pipe2_exp.fit(doc, pipe2.predict_proba)
pipe2_exp.show_prediction(target_names= target)

In [ ]:
def print_prediction_1(doc):
    print(doc)
    y_pred = pipe2.predict_proba([doc])[0]
    tar =['Reject','Accept']
    for target, prob in zip(tar, y_pred):
        print("{:.3f} {}".format(prob, target))

In [ ]:
pipe2_exp.explain_weights(target_names=target)

In [ ]:
print(pipe2.classes_) 
print(pipe2_exp.metrics_ )

In [ ]:
class Padding_2(BaseEstimator, TransformerMixin):
    
    def __init__(self,model):
        self.model = model # Configure model to return attention values
        self.tokenizer = tokenizer
        self.mxlenght = 400
        print('\n>>>>>>>init() called.\n')

    def fit(self, X, y = None):
        print('\n>>>>>>>fit() called.\n')
        return self
    

    def transform(self, X, y = None):
        embeddings = []
        for i in X:
            temp = [] 
            sentence_list = i.split(".")
            for i in sentence_list:
                if len(i)==0:
                    continue
                pred = model.predict(i)
                temp.append(pred)
            size = self.mxlenght - len(temp)
            if size > 0:
                temp.extend([-1]*size)
            elif size < 0:
                temp = temp[0:self.mxlenght+1]
            else:
                pass
            embeddings.append(temp)
        return embeddings

In [ ]:
pipe3 = Pipeline(steps=[
                       ('Documnet Embeddings', Padding_2(pipe2)), # this will trigger a call to __init__
                       ('Logistic Regression', LogisticRegression(solver='lbfgs')),

])

pipe3.fit(doc_train_text, doc_train_status)
pipe3.score(doc_test_text,doc_test_status)

In [ ]:
def print_prediction_2(doc):
    print(doc)
    y_pred = pipe3.predict_proba([doc])[0]
    tar =['Reject','Accept']
    for target, prob in zip(tar, y_pred):
        print("{:.3f} {}".format(prob, target))

In [ ]:
target =['Reject','Accept']
doc = text[0]
pipe3_exp = TextExplainer(random_state=42)
pipe3_exp.fit(doc, pipe3.predict_proba)
pipe3_exp.show_prediction(target_names= target)

In [ ]:
weights = pipe3_exp.explain_weights(top= None)
positivie_words= {}

for i in weights.targets[0].feature_weights.pos:
    #print(i.feature)
    g = positivie_words.get(i.feature,-1)
    if g==-1:
        positivie_words[i.feature]=1
    else:
        positivie_words[i.feature]+=1
        

negative_words= {}

for i in weights.targets[0].feature_weights.neg:
    #print(i.feature)
    g = negative_words.get(i.feature,-1)
    if g==-1:
        negative_words[i.feature]=1
    else:
        negative_words[i.feature]+=1
# print(weights.targets[0].feature_weights.pos[0].weight,weights.targets[0].feature_weights.pos[0].feature)

In [ ]:
wc = WordCloud(background_color="white",width=1000,height=1000,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(positivie_words)
plt.imshow(wc)

In [ ]:
wc = WordCloud(background_color="white",width=1000,height=1000,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(negative_words)
plt.imshow(wc)